In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta
import matplotlib as mpl
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
from datetime import datetime, timedelta
import warnings
import datetime
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)
mpl.style.use('seaborn')
pd.set_option('display.max_columns', 1000)
from sklearn.metrics import accuracy_score
import os

In [101]:
print(os.getcwd())
os.chdir(r"C:\Users\krchakravarthy\Documents\Forex_analysis\24_4_19")

C:\Users\krchakravarthy\Documents\Forex_analysis\24_4_19


In [102]:
df=pd.read_csv("aud_usd_h4_h1_all_merged_corr_0.1_v1.csv")
df.columns = df.columns.str.replace('.','_')

In [103]:
df.head()

Unnamed: 0                 time_local  target1_x  target2_x  target3_x  \
0           0  2010-01-05 18:00:00+00:00     0.9117     0.9135     0.9142   
1           1  2010-01-05 22:00:00+00:00     0.9135     0.9142     0.9142   
2           2  2010-01-06 02:00:00+00:00     0.9142     0.9142     0.9156   
3           3  2010-01-06 06:00:00+00:00     0.9142     0.9156     0.9184   
4           4  2010-01-06 10:00:00+00:00     0.9156     0.9184     0.9196   

   bid_close_x  Volatility_USD_hourly  PC_macro_economic_ind_USD_hourly  \
0       0.9124                    0.0                          0.000000   
1       0.9117                    0.5                         -1.543434   
2       0.9135                    0.5                         -1.543434   
3       0.9142                    0.5                         -1.543434   
4       0.9142                    0.5                         -1.543434   

   Volatility_USD_daily  PC_macro_economic_ind_USD_daily  \
0                   1.0                         0.051546   
1                   1.0                         0.051546   
2                   1.0                        -1.050000   
3                   1.0                        -1.050000   
4                   1.0                        -1.050000   

   Volatility_AUD_hourly  PC_macro_economic_ind_AUD_hourly  \
0                    0.0                          0.000000   
1                    0.5                         -1.543434   
2                    0.5                         -1.543434   
3                    0.5                         -1.543434   
4                    0.5                         -1.543434   

   Volatility_AUD_daily  PC_macro_economic_ind_AUD_daily  \
0                   1.0                         0.051546   
1                   1.0                         0.051546   
2                   1.0                        -1.050000   
3                   1.0                        -1.050000   
4                   1.0                        -1.050000   

   ANZ Job Advertisements_PC_AUD_hourly  \
0                                   0.0   
1                                   0.0   
2                                   0.0   
3                                   0.0   
4                                   0.0   

   AiG Performance of Mfg Index_PC_AUD_hourly  \
0                                         0.0   
1                                         0.0   
2                                         0.0   
3                                         0.0   
4                                         0.0   

   Building Permits (MoM)_PC_AUD_hourly  Building Permits (YoY)_PC_AUD_hourly  \
0                              0.000000                              0.000000   
1                             -4.484536                              1.397668   
2                             -4.484536                              1.397668   
3                             -4.484536                              1.397668   
4                             -4.484536                              1.397668   

   CFTC AUD NC net positions_PC_AUD_hourly  \
0                                      0.0   
1                                      0.0   
2                                      0.0   
3                                      0.0   
4                                      0.0   

   Construction Work Done_PC_AUD_hourly  \
0                                   0.0   
1                                   0.0   
2                                   0.0   
3                                   0.0   
4                                   0.0   

   Consumer Price Index (QoQ)_PC_AUD_hourly  \
0                                       0.0   
1                                       0.0   
2                                       0.0   
3                                       0.0   
4                                       0.0   

   Dwelling Unit Starts (QoQ)_PC_AUD_hourly  \
0                                       0.0   
1                                       

In [104]:
df=df.dropna(how='all',axis=1)
df.shape

(14028, 839)

In [105]:
df['year']=pd.DatetimeIndex(df['time_local']).year
df['month']=pd.DatetimeIndex(df['time_local']).month
df['day']=pd.DatetimeIndex(df['time_local']).day
df['hour']=pd.DatetimeIndex(df['time_local']).hour
df['dayofweek']=pd.DatetimeIndex(df['time_local']).dayofweek



In [106]:
import datetime   
#date = datetime.date.today()
date=pd.datetime.now().strftime("%Y-%m-%d %I:%M:%S")
date

'2019-04-24 06:07:05'

In [107]:
df['today']=date
df['today'] =pd.DatetimeIndex(df['today'])

In [108]:
df['time_local'].head()

0    2010-01-05 18:00:00+00:00
1    2010-01-05 22:00:00+00:00
2    2010-01-06 02:00:00+00:00
3    2010-01-06 06:00:00+00:00
4    2010-01-06 10:00:00+00:00
Name: time_local, dtype: object

In [109]:
df['nb_months'] = (df['today'] - pd.to_datetime(df['time_local'].astype(str).str[:-6]))/np.timedelta64(1, 'M')
df['nb_months'] = df['nb_months'].astype(int)

In [110]:
col_drop= df.filter(regex='time').columns

In [111]:
target_cols = df.filter(regex='target').columns

In [112]:
cols_x = df.filter(regex='_x').columns

In [113]:
cols_y = df.filter(regex='_y').columns

In [114]:
X=df.drop(['Unnamed: 0','time_local', 'target2_x', 'target3_x', 'target1_y', 'target2_y',
       'target3_y', 'class_target2', 'class_target3',
       'PC_AUD_H4_target1', 'bid_close_y', 'today','target1_x'],axis=1)

In [115]:
time_local=df.time_local

In [116]:
X=X.fillna(0)

In [117]:
X.head()

bid_close_x  Volatility_USD_hourly  PC_macro_economic_ind_USD_hourly  \
0       0.9124                    0.0                          0.000000   
1       0.9117                    0.5                         -1.543434   
2       0.9135                    0.5                         -1.543434   
3       0.9142                    0.5                         -1.543434   
4       0.9142                    0.5                         -1.543434   

   Volatility_USD_daily  PC_macro_economic_ind_USD_daily  \
0                   1.0                         0.051546   
1                   1.0                         0.051546   
2                   1.0                        -1.050000   
3                   1.0                        -1.050000   
4                   1.0                        -1.050000   

   Volatility_AUD_hourly  PC_macro_economic_ind_AUD_hourly  \
0                    0.0                          0.000000   
1                    0.5                         -1.543434   
2                    0.5                         -1.543434   
3                    0.5                         -1.543434   
4                    0.5                         -1.543434   

   Volatility_AUD_daily  PC_macro_economic_ind_AUD_daily  \
0                   1.0                         0.051546   
1                   1.0                         0.051546   
2                   1.0                        -1.050000   
3                   1.0                        -1.050000   
4                   1.0                        -1.050000   

   ANZ Job Advertisements_PC_AUD_hourly  \
0                                   0.0   
1                                   0.0   
2                                   0.0   
3                                   0.0   
4                                   0.0   

   AiG Performance of Mfg Index_PC_AUD_hourly  \
0                                         0.0   
1                                         0.0   
2                                         0.0   
3                                         0.0   
4                                         0.0   

   Building Permits (MoM)_PC_AUD_hourly  Building Permits (YoY)_PC_AUD_hourly  \
0                              0.000000                              0.000000   
1                             -4.484536                              1.397668   
2                             -4.484536                              1.397668   
3                             -4.484536                              1.397668   
4                             -4.484536                              1.397668   

   CFTC AUD NC net positions_PC_AUD_hourly  \
0                                      0.0   
1                                      0.0   
2                                      0.0   
3                                      0.0   
4                                      0.0   

   Construction Work Done_PC_AUD_hourly  \
0                                   0.0   
1                                   0.0   
2                                   0.0   
3                                   0.0   
4                                   0.0   

   Consumer Price Index (QoQ)_PC_AUD_hourly  \
0                                       0.0   
1                                       0.0   
2                                       0.0   
3                                       0.0   
4                                       0.0   

   Dwelling Unit Starts (QoQ)_PC_AUD_hourly  \
0                                       0.0   
1                                       0.0   
2                                       0.0   
3                                       0.0   
4                                       0.0   

   Employment Change s_a__PC_AUD_hourly  \
0                                   0.0   
1                                   0.0   
2                                   0.0   
3                                   0.0   
4                                   0.0   

   Export Price Index (QoQ)_PC_AUD_hourly  Exports_PC_A

In [118]:
df_temp=X.select_dtypes(include=['object'])

In [119]:
obj=df_temp.loc[:, df_temp.dtypes =='object']
for col in obj:
    obj[col] = pd.to_numeric(obj[col], errors='coerce')
df_temp[obj.columns]=obj

In [120]:
df_temp=df_temp.replace([np.inf, -np.inf], np.nan)
df_temp=df_temp.fillna(0)
X[df_temp.columns]=df_temp

In [121]:
#X_final= X[X['year'] < 2018]
s=X[['year','month', 'dayofweek']]
for each in s:
    dummies = pd.get_dummies(s[each], prefix=each, drop_first=False)
    X=X.drop([each],axis=1)
    X = pd.concat([X, dummies], axis=1)

In [122]:
X=X[X.replace([np.inf, -np.inf], np.nan).notnull().all(axis=1)]

In [123]:
X['target_2_class_1']=np.where(X['class_target1']==-1,-1,0)
X['target_2_class_2']=np.where(X['class_target1']==1,1,0)

In [124]:
all_col=X.columns
drop_col=list(x for x in all_col if 'h1_5' in x or 'h1_6' in x or 'h1_7' in x  )
X=X.drop(columns=drop_col,axis=1)

In [125]:
X_final=X[(X['year_2019']!=1) & (X['year_2018']!=1) ]

In [126]:
X_pred=X[(X['year_2019']==1) | (X['year_2018']==1)]
X_pred.to_csv("X_class_target1_2018_23_4_2019_two_class.csv")

In [127]:
y=X_final['target_2_class_1']
X1=X_final.drop(['target_2_class_1','target_2_class_2','class_target1'],axis=1)

In [58]:
# from sklearn.preprocessing import StandardScaler
#scaler=StandardScaler()
#scaler.fit(X1)
#X1=scaler.transform(X1)

In [59]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
np.seed=0
from sklearn.linear_model import LogisticRegression

In [128]:
from imblearn.over_sampling import SMOTE
smt = SMOTE(random_state=1)
X2, y1 = smt.fit_sample(X1, y)
len(X2),len(y1)
X2=pd.DataFrame(X2)
y1=pd.DataFrame(y1)
X2.columns=X1.columns

In [129]:
X_train, X_test, y_train, y_test= train_test_split(X2, y1,test_size=0.3,random_state=1,stratify=y1)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)


(9788, 832) (9788, 1)
(4196, 832) (4196, 1)


In [62]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier()
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)

In [63]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.58      0.75      0.65      2098
           0       0.65      0.46      0.54      2098

   micro avg       0.60      0.60      0.60      4196
   macro avg       0.61      0.60      0.59      4196
weighted avg       0.61      0.60      0.59      4196



In [64]:
print(classification_report(y_train,gb.predict(X_train)))

              precision    recall  f1-score   support

          -1       0.68      0.86      0.76      4894
           0       0.81      0.59      0.69      4894

   micro avg       0.73      0.73      0.73      9788
   macro avg       0.75      0.73      0.72      9788
weighted avg       0.75      0.73      0.72      9788



In [65]:
accuracy_score(y_test,y_pred)

0.6034318398474738

In [66]:
accuracy_score(y_train,gb.predict(X_train))

0.7288516550878626

In [68]:
gb

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [70]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.8,n_estimators=80)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)

In [71]:
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.5993803622497617, 0.639967306906416)

In [ ]:

from sklearn.model_selection import GridSearchCV
parameters = {
    "learning_rate": [0.1],
    "max_depth":[3,5,8],
    "criterion": ["friedman_mse"],
    "n_estimators":[90,100,120],
    
    "subsample":[0.9,1.0]
}

clf = GridSearchCV(gb, parameters, cv=5,verbose=5)
clf.fit(X_train,y_train)

In [132]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.8,n_estimators=90)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.6034318398474738, 0.6488557417245607)

In [133]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.58      0.75      0.65      2098
           0       0.65      0.46      0.53      2098

   micro avg       0.60      0.60      0.60      4196
   macro avg       0.61      0.60      0.59      4196
weighted avg       0.61      0.60      0.59      4196



In [134]:
from joblib import dump, load
dump(gb, 'target_2_class_1_grad.joblib') 

['target_2_class_1_grad.joblib']

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.8,n_estimators=100)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.8,n_estimators=110)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.8,n_estimators=110,max_depth=4)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.8,n_estimators=120,max_depth=4)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

In [ ]:
#max_features="auto"
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.85,n_estimators=130,max_depth=4,max_features="auto")
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

# Final_model

In [73]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,model.predict(X_train))

(0.6182078169685414, 0.7258888434818145)

In [74]:
from sklearn.metrics import classification_report

In [75]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.59      0.77      0.67      2098
           0       0.67      0.47      0.55      2098

   micro avg       0.62      0.62      0.62      4196
   macro avg       0.63      0.62      0.61      4196
weighted avg       0.63      0.62      0.61      4196



In [76]:
print(classification_report(y_train,model.predict(X_train)))

              precision    recall  f1-score   support

          -1       0.68      0.87      0.76      4894
           0       0.82      0.58      0.68      4894

   micro avg       0.73      0.73      0.73      9788
   macro avg       0.75      0.73      0.72      9788
weighted avg       0.75      0.73      0.72      9788



In [77]:
feature_importance=pd.DataFrame(model.feature_importances_)
fetaure_cols=pd.DataFrame(X1.columns)
feature_df=pd.concat([fetaure_cols,feature_importance],axis=1)
feature_df.columns=['col_name','value']
feature_imp_df=feature_df.sort_values(by='value',ascending=False)

In [78]:
features_non_zero=feature_imp_df[feature_imp_df['value']>0.0015]['col_name']

In [79]:
len(features_non_zero)

220

In [80]:
feature_imp_df.to_csv("feature_imp_fx.csv")

In [81]:
X2_1=X2[features_non_zero]
X2_1=pd.DataFrame(X2_1)
y1_1=pd.DataFrame(y1)
#X2_1.columns=X2_1.columns

In [82]:
cols_year=X2_1.filter(regex='year').columns
X2_1=X2_1.drop(cols_year,axis=1)

In [83]:

X_train, X_test, y_train, y_test= train_test_split(X2_1, y1_1,test_size=0.3,random_state=1,stratify=y1)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)


(9788, 214) (9788, 1)
(4196, 214) (4196, 1)


In [84]:
from xgboost import XGBClassifier
model = XGBClassifier(reg_lambda=1)
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,model.predict(X_train))

(0.6146329837940896, 0.7269105026563139)

In [85]:
from sklearn.metrics import classification_report

In [86]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.59      0.76      0.66      2098
           0       0.66      0.47      0.55      2098

   micro avg       0.61      0.61      0.61      4196
   macro avg       0.62      0.61      0.61      4196
weighted avg       0.62      0.61      0.61      4196



In [139]:
print(classification_report(y_train,model.predict(X_train)))

              precision    recall  f1-score   support

          -1       0.60      0.90      0.72      4253
           0       0.79      0.39      0.52      4253

   micro avg       0.64      0.64      0.64      8506
   macro avg       0.69      0.64      0.62      8506
weighted avg       0.69      0.64      0.62      8506



In [140]:
from joblib import dump, load
dump(model, 'xgb_target_2_class_1.joblib') 

['xgb_target_2_class_1.joblib']

In [83]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,y_pred))

[[1382  441]
 [1035  788]]


In [135]:
y=X_final['target_2_class_2']
X1=X_final.drop(['target_2_class_1','target_2_class_2','class_target1'],axis=1)

In [136]:
X_train, X_test, y_train, y_test= train_test_split(X1, y,test_size=0.3,random_state=1,stratify=y)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(8459, 832) (8459,)
(3626, 832) (3626,)


In [137]:
from xgboost import XGBClassifier
model = XGBClassifier(reg_lambda=1)
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,model.predict(X_train))

(0.5970766685052399, 0.6643811325215746)

In [138]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.61      0.95      0.74      2191
           1       0.44      0.06      0.11      1435

   micro avg       0.60      0.60      0.60      3626
   macro avg       0.52      0.50      0.42      3626
weighted avg       0.54      0.60      0.49      3626



In [139]:
print(classification_report(y_train,model.predict(X_train)))

              precision    recall  f1-score   support

           0       0.65      0.99      0.78      5110
           1       0.89      0.17      0.29      3349

   micro avg       0.66      0.66      0.66      8459
   macro avg       0.77      0.58      0.53      8459
weighted avg       0.74      0.66      0.59      8459



In [140]:
dump(model, 'xgb_target_2_class_2.joblib')

['xgb_target_2_class_2.joblib']

In [94]:
feature_importance=pd.DataFrame(model.feature_importances_)
fetaure_cols=pd.DataFrame(X1.columns)
feature_df=pd.concat([fetaure_cols,feature_importance],axis=1)
feature_df.columns=['col_name','value']
feature_imp_df=feature_df.sort_values(by='value',ascending=False)

In [95]:
feature_imp_df=pd.read_csv("feature_imp_fx.csv")
features_non_zero=feature_imp_df[feature_imp_df['value']>0.0015]['col_name']
X2_1=X1[features_non_zero]
X2_1=pd.DataFrame(X1)
y1_1=pd.DataFrame(y)
#X2_1.columns=X2_1.columns


In [96]:
X_train, X_test, y_train, y_test= train_test_split(X2_1, y1_1,test_size=0.3,random_state=1,stratify=y)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(8459, 832) (8459, 1)
(3626, 832) (3626, 1)


In [97]:
from xgboost import XGBClassifier
model = XGBClassifier(reg_lambda=1)
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,model.predict(X_train))

(0.597904026475455, 0.6640264806714742)

In [98]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.61      0.95      0.74      2191
           1       0.44      0.06      0.11      1435

   micro avg       0.60      0.60      0.60      3626
   macro avg       0.52      0.51      0.42      3626
weighted avg       0.54      0.60      0.49      3626



In [99]:
dump(model, 'xgb_target_2_class_2_0_2.joblib')

NameError: name 'dump' is not defined

In [ ]:
#feature_imp_df.to_csv("feature_imp_fx.csv")